# Оптимизация выполнения кода, векторизация, Numba

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* IPython Cookbook, Second Edition (2018), глава 4
* https://ipython-books.github.io/43-profiling-your-code-line-by-line-with-line_profiler/
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html
* https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

## Задачи для совместного разбора

In [ ]:
!pip install git+https://github.com/pyutils/line_profiler.git
!pip install line_profiler
!pip install --user numpy==1.20

In [1]:
import numpy as np, pandas as pd
import random, numba, string, re, json
from numba import jit, njit, vectorize, float64
from typing import Union

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
A = np.array([random.randint(0, 1000) for _ in range(0, 1000000)])
B = A + 100
B.mean()

599.767969

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Примените функцию к каждому элементу массива.

In [3]:
def summ_all(x):
    return sum(x[:-1])
print(A)
summ_all(A)

[343 321 780 ... 775 855 471]


499767498

In [4]:
%timeit summ_all(A)

85.1 ms ± 9.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


3. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [5]:
N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=['col1', 'col2', 'col3', 'col4'])
df['key'] = np.random.choice(list(string.ascii_lowercase), N, replace=True)
df[df['key'].isin([chr(i) for i in range(97, 102)])]

,col1,col2,col3,col4,key
3,-1.103704,-1.401043,0.105232,-0.412023,c
12,-0.254321,-0.288417,-0.322897,1.207355,d
15,-0.871431,-0.994775,0.607333,-0.943332,b
16,-0.929123,0.079503,-0.569747,-0.607858,e
18,0.436897,0.294763,1.696682,0.655834,d
...,...,...,...,...,...
1999986,-0.743301,-0.475192,0.179314,0.824560,c
1999992,-2.054903,1.565012,-1.143717,-1.154886,a
1999997,1.148695,0.735811,-1.440515,1.172578,a
1999998,0.639840,-2.080992,0.742642,0.745725,b


In [6]:
[chr(i) for i in range(97, 102)]

['a', 'b', 'c', 'd', 'e']

## Лабораторная работа 8

В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

In [7]:
recipes = pd.read_csv('data/recipes_sample.csv', sep=',', parse_dates=['submitted'])
reviews = pd.read_csv('data/reviews_sample.csv', sep=',', parse_dates=['date'])
reviews.set_index("Unnamed: 0", inplace=True)
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [8]:
reviews.head()

,user_id,recipe_id,date,rating,review
Unnamed: 0,,,,,
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [9]:
recipes.dtypes

name                      object
id                         int64
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
n_steps                  float64
description               object
n_ingredients            float64
dtype: object

In [10]:
reviews.dtypes

user_id               int64
recipe_id             int64
date         datetime64[ns]
rating                int64
review               object
dtype: object

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [11]:
recipes = recipes.drop(recipes[recipes['submitted'].dt.year != 2010].index)

In [12]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    summ_len, count = 0, 0
    for _, row in df.iterrows():
        summ_len += len(row['name']) + len(row['description']) + 1 # пробел
        count += 1
    return summ_len / count

In [13]:
get_mean_len_A(recipes)

265.501300390117

1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [14]:
def get_mean_len_B(df: pd.DataFrame) -> float:
  return df.apply(lambda row: row['name'] + ' ' + row['description'], axis = 1).str.len().mean()

In [15]:
get_mean_len_B(recipes)

265.501300390117

1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи метода серий получаете среднюю длину полных описаний.

In [16]:
def get_mean_len_C(df: pd.DataFrame):
    ser_des = df['name'] + ' ' + df['description']
    len_s = ser_des.str.len()
    return len_s.mean()

In [17]:
get_mean_len_C(recipes)

265.501300390117

1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [18]:
get_mean_len_A(recipes) == get_mean_len_B(recipes) == get_mean_len_C(recipes)

True

In [19]:
%timeit get_mean_len_A(recipes)

77.3 ms ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%timeit get_mean_len_B(recipes)

26.8 ms ± 5.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%timeit get_mean_len_C(recipes)

2.01 ms ± 247 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%time get_mean_len_A(recipes)

Wall time: 72 ms


265.501300390117

In [23]:
%time get_mean_len_B(recipes)

Wall time: 40 ms


265.501300390117

In [24]:
%time get_mean_len_C(recipes)

Wall time: 0 ns


265.501300390117

## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово.

2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально.

In [25]:
%load_ext line_profiler
def get_word_reviews_count(df):
    word_reviews = {}

    for _, row in df.dropna(subset=["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])

    return word_reviews_count
%lprun -f get_word_reviews_count get_word_reviews_count(reviews.head(10000))

Timer unit: 1e-09 s

Total time: 3.52708 s
File: <ipython-input-9-3d8ba416c14a>
Function: get_word_reviews_count at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def get_word_reviews_count(df):
     3         1        833.0    833.0      0.0      word_reviews = {}
     4
     5      9999 1032153538.0 103225.7     29.3      for _, row in df.dropna(subset=["review"]).iterrows():
     6      9999  198787164.0  19880.7      5.6          recipe_id, review = row["recipe_id"], row["review"]
     7      9999  160701643.0  16071.8      4.6          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
     8    530263  103969488.0    196.1      2.9          for word in words:
     9    510935  205680527.0    402.6      5.8              if word not in word_reviews:
    10     19328    9665153.0    500.1      0.3                  word_reviews[word] = []
    11    530263  268548776.0    506.4      7.6              word_reviews[word].append(recipe_id)
    12
    13         1        240.0    240.0      0.0      word_reviews_count = {}
    14      9999  838388276.0  83847.2     23.8      for _, row in df.dropna(subset=["review"]).iterrows():
    15      9999  110387306.0  11039.8      3.1          review = row["review"]
    16      9999  146240136.0  14625.5      4.1          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
    17    530263  111374835.0    210.0      3.2          for word in words:
    18    530263  341180923.0    643.4      9.7              word_reviews_count[word] = len(word_reviews[word])
    19
    20         1        181.0    181.0      0.0      return word_reviews_count

##### В описанной выше функции применяется цикл в цикле, что не есть что хорошо, поскольку это — лишняя трата ресурсов устройства и трата времени + бессмысленно делать такую процедуру дважды (идёт дополнительная нагрузка на оперативную память).


2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [26]:
def get_word_reviews_count_update(df):
  serr = df['review'].apply(lambda row: np.unique(re.sub('[^A-Za-z\s]', '', str(row)).split()))
  return dict(pd.value_counts(np.hstack(serr)))
%lprun -f get_word_reviews_count_update get_word_reviews_count_update(reviews.head(10000))

Timer unit: 1e-09 s

Total time: 0.737537 s
File: <ipython-input-8-4d481325ae6b>
Function: get_word_reviews_count_update at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def get_word_reviews_count_update(df):
     3         1  422681235.0 422681235.0     57.3    serr = df['review'].apply(lambda row: np.unique(re.sub('[^A-Za-z\s]', '', str(row)).split()))
     4         1  314855368.0 314855368.0     42.7    return dict(pd.value_counts(np.hstack(serr)))

## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

In [27]:
with open('data/rating_predictions.json', 'r', encoding='utf-8') as file:
    rating_pred = pd.DataFrame(json.load(file))
rating_pred

,rating,prediction
0,5.0,4.944444
1,5.0,4.437500
2,5.0,4.727273
3,5.0,4.354545
4,5.0,4.888889
...,...,...
119886,5.0,4.903226
119887,5.0,4.333333
119888,5.0,5.000000
119889,5.0,4.142857


3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.


In [28]:
def mape_lists(A_list, F_list):
  n, mape = len(A_list), 0
  for i in range(n):
    mape += abs((A_list[i] - F_list[i]) / F_list[i])
  return (100 * mape) / n

In [29]:
A_list, F_list = rating_pred['rating'], rating_pred['prediction']
mape_lists(A_list, F_list)

11.949451393825532

In [30]:
%timeit mape_lists(A_list, F_list)

1.04 s ± 223 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%time mape_lists(A_list, F_list)

Wall time: 864 ms


11.949451393825532

3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [32]:
def mape_numpy(A_array, F_array):
  return (np.sum(abs((A_array - F_array) / F_array)) * 100) / len(A_list)

In [33]:
A_array, F_array = np.array(A_list), np.array(F_list)
mape_numpy(A_array, F_array)

11.949451393824203

In [34]:
%timeit mape_numpy(A_array, F_array)

1.08 ms ± 91.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [35]:
%time mape_numpy(A_array, F_array)

Wall time: 0 ns


11.949451393824203

3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`.

In [36]:
@njit
def mape_numba(A_typed, F_typed):
  n, mape = len(A_typed), 0
  for i in range(n):
    mape += abs((A_typed[i] - F_typed[i]) / F_typed[i])
  return 100 * mape / n

In [37]:
A_typed, F_typed = numba.typed.List(A_list), numba.typed.List(F_list)
mape_numba(A_typed,F_typed)

11.949451393825532

In [38]:
%timeit mape_numba(A_typed,F_typed)

1.16 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [39]:
%timeit mape_numba(A_array,F_array)

228 µs ± 38.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [40]:
%time mape_numba(A_typed, F_typed)

Wall time: 0 ns


11.949451393825532

In [41]:
%time mape_numba(A_array, F_array)

Wall time: 0 ns


11.949451393825532

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [42]:
def is_pretty(number: int):
  first = number
  while first >= 10:
    first = first // 10
  if first == (number % 10):
    return True
  else:
    return False

In [43]:
is_pretty(3022678)

False

In [44]:
is_pretty(3045098096343)

True

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

In [45]:
ids = reviews["recipe_id"].values

In [46]:
sum(list(map(is_pretty, ids)))

11955

In [47]:
%timeit sum(list(map(is_pretty, ids)))

277 ms ± 9.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%time sum(list(map(is_pretty, ids)))

Wall time: 264 ms


11955

4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [49]:
v_is_pretty = np.vectorize(is_pretty)
v_is_pretty(ids).sum()

11955

In [50]:
%timeit v_is_pretty(ids).sum()

78.5 ms ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
%time v_is_pretty(ids).sum()

Wall time: 118 ms


11955

4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [52]:
is_pretty_numba = numba.vectorize(is_pretty)
is_pretty_numba(ids).sum()

11955

In [53]:
%timeit is_pretty_numba(ids).sum()

1.35 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [54]:
%time is_pretty_numba(ids).sum()

Wall time: 7.97 ms


11955